overview of this notebook...

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import xskillscore as xs
%load_ext blackcellmagic

Use the same data as in 01_Deterministic.py

In [2]:
stores = np.arange(4)
skus = np.arange(3)
dates = pd.date_range("1/1/2020", "1/5/2020", freq="D")

rows = []
for _, date in enumerate(dates):
    for _, store in enumerate(stores):
        for _, sku in enumerate(skus):
            rows.append(
                dict(
                    {
                        "DATE": date,
                        "STORE": store,
                        "SKU": sku,
                        "QUANTITY_SOLD": np.random.randint(10),
                    }
                )
            )
df = pd.DataFrame(rows)
df.rename(columns={"QUANTITY_SOLD": "y"}, inplace=True)
df.set_index(['DATE', 'STORE', 'SKU'], inplace=True)
df.head()

y
DATE       STORE SKU   
2020-01-01 0     0    3
                 1    2
                 2    7
           1     0    1
                 1    6

Make multiple predictions...

Append the predictions to the df using a `member` column...

In [3]:
tmp = df.copy()
for i in range(1, 7):
    tmp['member'] = i
    noise = np.random.uniform(-1, 1, size=len(df['y']))
    tmp['yhat'] = (df['y'] + (df['y'] * noise)).astype(int)
    if i == 1:
        df_yhat = tmp.copy()
    else:
        df_yhat = df_yhat.append(tmp)
df_yhat

y  member  yhat
DATE       STORE SKU                 
2020-01-01 0     0    3       1     2
                 1    2       1     0
                 2    7       1     2
           1     0    1       1     0
                 1    6       1     8
...                  ..     ...   ...
2020-01-05 2     1    3       6     2
                 2    2       6     1
           3     0    3       6     5
                 1    2       6     3
                 2    2       6     0

[360 rows x 3 columns]

Drop the `y` column and add `member` to the index...

In [4]:
df_yhat.drop('y', axis=1, inplace=True)
df_yhat.set_index(['member'], append=True, inplace=True)
df_yhat

yhat
DATE       STORE SKU member      
2020-01-01 0     0   1          2
                 1   1          0
                 2   1          2
           1     0   1          0
                 1   1          8
...                           ...
2020-01-05 2     1   6          2
                 2   6          1
           3     0   6          5
                 1   6          3
                 2   6          0

[360 rows x 1 columns]

Convert the target dataframe to an xarray.Dataset... 

In [5]:
ds = df.to_xarray()
ds

<xarray.Dataset>
Dimensions:  (DATE: 5, SKU: 3, STORE: 4)
Coordinates:
  * DATE     (DATE) datetime64[ns] 2020-01-01 2020-01-02 ... 2020-01-05
  * STORE    (STORE) int64 0 1 2 3
  * SKU      (SKU) int64 0 1 2
Data variables:
    y        (DATE, STORE, SKU) int64 3 2 7 1 6 3 5 1 9 5 ... 2 3 6 9 3 2 3 2 2

Add the predicted dataframe as an `xarray.DataArray` to the `xarray.Dataset`

In [6]:
ds['yhat'] = df_yhat.to_xarray()['yhat']
ds

<xarray.Dataset>
Dimensions:  (DATE: 5, SKU: 3, STORE: 4, member: 6)
Coordinates:
  * DATE     (DATE) datetime64[ns] 2020-01-01 2020-01-02 ... 2020-01-05
  * STORE    (STORE) int64 0 1 2 3
  * SKU      (SKU) int64 0 1 2
  * member   (member) int64 1 2 3 4 5 6
Data variables:
    y        (DATE, STORE, SKU) int64 3 2 7 1 6 3 5 1 9 5 ... 2 3 6 9 3 2 3 2 2
    yhat     (DATE, STORE, SKU, member) int64 2 1 1 1 0 1 0 3 ... 3 2 3 1 3 0 0

# Using xskillscore - CRPS

What is CRPS...

It's useage in kaggle comps...

Apply CRPS over the member dimension... This will return the CRPS as a date, store and sku level

In [7]:
ds.xs.crps_ensemble('y', 'yhat', dim='member')

<xarray.DataArray (DATE: 5, STORE: 4, SKU: 3)>
array([[[1.72222222, 0.58333333, 2.5       ],
        [0.44444444, 1.25      , 0.5       ],
        [0.55555556, 0.11111111, 0.77777778],
        [0.97222222, 1.61111111, 1.72222222]],

       [[1.30555556, 1.63888889, 0.88888889],
        [1.83333333, 0.25      , 1.41666667],
        [1.13888889, 1.72222222, 0.11111111],
        [0.11111111, 1.47222222, 0.44444444]],

       [[0.        , 0.77777778, 0.83333333],
        [0.25      , 1.69444444, 0.38888889],
        [2.25      , 2.11111111, 0.        ],
        [0.30555556, 3.05555556, 1.16666667]],

       [[1.86111111, 1.13888889, 0.75      ],
        [1.27777778, 0.25      , 0.66666667],
        [0.02777778, 1.19444444, 0.47222222],
        [2.44444444, 0.94444444, 0.38888889]],

       [[0.72222222, 0.47222222, 0.        ],
        [0.13888889, 1.13888889, 1.11111111],
        [4.22222222, 0.75      , 0.47222222],
        [1.        , 0.16666667, 0.47222222]]])
Coordinates:
  * DATE     (DATE) datetime64[ns] 2020-01-01 2020-01-02 ... 2020-01-05
  * STORE    (STORE) int64 0 1 2 3
  * SKU      (SKU) int64 0 1 2

To return an overal CRPS it is recommened averaging over all dimensions before using `crps`...

In [8]:
y = ds['y'].mean(dim=['DATE', 'STORE', 'SKU'])
print(y)
yhat = ds['yhat'].mean(dim=['DATE', 'STORE', 'SKU'])
print(yhat)
xs.crps_ensemble(y, yhat, dim='member')

<xarray.DataArray 'y' ()>
array(4.21666667)
<xarray.DataArray 'yhat' (member: 6)>
array([3.48333333, 3.56666667, 4.08333333, 4.13333333, 3.68333333,
       3.41666667])
Coordinates:
  * member   (member) int64 1 2 3 4 5 6


<xarray.DataArray ()>
array(0.33611111)